In [1]:
!pip install torch transformers datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import os
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer

# import pytorch_lightning as pl
import warnings
warnings.simplefilter("ignore", UserWarning)


In [3]:
if not os.path.exists("/content/cleaned_data_version2.zip"):
  !gdown --id 1ARLmZTC_gmM4_3eT7vv7n2ub1hD0TR9n

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ARLmZTC_gmM4_3eT7vv7n2ub1hD0TR9n
From (redirected): https://drive.google.com/uc?id=1ARLmZTC_gmM4_3eT7vv7n2ub1hD0TR9n&confirm=t&uuid=4e042cb4-0b2e-429d-b797-4a3446276668
To: /content/cleaned_data_version2.zip
100% 33.1M/33.1M [00:00<00:00, 75.5MB/s]


# Load Data

In [4]:
if not os.path.exists("/content/out"):
  !unzip /content/cleaned_data_version2.zip

Archive:  /content/cleaned_data_version2.zip
   creating: cleaned_data/
  inflating: cleaned_data/C1E001.json  
  inflating: cleaned_data/C1E002.json  
  inflating: cleaned_data/C1E003.json  
  inflating: cleaned_data/C1E004.json  
  inflating: cleaned_data/C1E005.json  
  inflating: cleaned_data/C1E006.json  
  inflating: cleaned_data/C1E007.json  
  inflating: cleaned_data/C1E008.json  
  inflating: cleaned_data/C1E009.json  
  inflating: cleaned_data/C1E010.json  
  inflating: cleaned_data/C1E011.json  
  inflating: cleaned_data/C1E013.json  
  inflating: cleaned_data/C1E014.json  
  inflating: cleaned_data/C1E015.json  
  inflating: cleaned_data/C1E016.json  
  inflating: cleaned_data/C1E017.json  
  inflating: cleaned_data/C1E018.json  
  inflating: cleaned_data/C1E019.json  
  inflating: cleaned_data/C1E020.json  
  inflating: cleaned_data/C1E021.json  
  inflating: cleaned_data/C1E022.json  
  inflating: cleaned_data/C1E023.json  
  inflating: cleaned_data/C1E024.json  
  inflat

In [5]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP')

Mounted at /content/drive


In [6]:
dfs = []

directory = '/content/out'
for entry in os.scandir(directory):
  if entry.is_file():
    path = entry.path
    df = pd.read_csv(path)
    dfs.append(df)

In [7]:
df = pd.concat(dfs)

In [8]:
df.shape

(398682, 3)

In [9]:
df.head()

,fileID,before_utterances,dm
0,C2E001,NaN,"<|begin_of_text|>Hello, everyone. (laughter)<|..."
1,C2E001,"<DM> <|begin_of_text|>Hello, everyone. (laught...",<|begin_of_text|>And welcome to tonight's epis...
2,C2E001,"<DM> <|begin_of_text|>Hello, everyone. (laught...",<|begin_of_text|>Super excited to be back. I'v...
3,C2E001,"<DM> <|begin_of_text|>Hello, everyone. (laught...",<|begin_of_text|><|end_of_text|>
4,C2E001,"<DM> <|begin_of_text|>Hello, everyone. (laught...","<|begin_of_text|>Nnnn. But yeah, so they're go..."


In [10]:
df = df.dropna()
df.shape

(398523, 3)

# Train Val Test Split

In [11]:
train_ids = pd.read_csv("/content/train_files", header=None).squeeze("columns")
val_ids = pd.read_csv("/content/val_files", header=None).squeeze("columns")
test_ids = pd.read_csv("/content/test_files", header=None).squeeze("columns")

In [12]:
train_df = df[df["fileID"].isin(train_ids)]
val_df = df[df["fileID"].isin(val_ids)]
test_df = df[df["fileID"].isin(test_ids)]

In [13]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(316393, 3)
(41930, 3)
(40200, 3)


# Prepare Model

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
# userdata.get('huggingFace')

In [15]:
from huggingface_hub import login
login()

In [23]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",  # Automatically map layers across GPUs if available
#     load_in_4bit=True   # Enables memory-efficient training
# )
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [24]:
example = train_df.iloc[100,1]
example

"<PLAYER4> <|begin_of_text|>Were we supposed to prepare this?<|end_of_text|>\n<PLAYER6> <|begin_of_text|>I'm pretty filthy. I have a mess of reddish-brown hair, and really filthy road clothes. I wear a long coat that I slept in. I slept about 20 hours last night. Jeez. Unshaven, a bit of a mess.<|end_of_text|>\n<PLAYER4> <|begin_of_text|>So far you're just talking about current Liam.<|end_of_text|>\n<PLAYER7> <|begin_of_text|>What color are your eyes? I need visual.<|end_of_text|>\n<PLAYER6> <|begin_of_text|>They're blue. That's it. It was a rough day yesterday, and-- that's it.<|end_of_text|>"

In [25]:
len(tokenizer(example)['input_ids'])

143

In [26]:
from torch.utils.data import Dataset

class DMNSPDataset(Dataset):
    def __init__(self, before_utterances, dm, tokenizer, max_length=256):
        self.before_utterances = before_utterances
        self.dm = dm
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dm)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.before_utterances[idx],
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            max_length=self.max_length
        )
        labels = self.tokenizer(
            self.before_utterances[idx]+self.dm[idx],
            return_tensors='pt',
            truncation=True,
            padding='max_length',
            max_length=self.max_length
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": labels['input_ids'].squeeze(0)
        }


In [27]:
train_dataset = DMNSPDataset(train_df['before_utterances'].to_list(), train_df['dm'].to_list(), tokenizer)
tmp = val_df.sample(1000)
val_dataset = DMNSPDataset(tmp['before_utterances'].to_list(), tmp['dm'].to_list(), tokenizer)
test_dataset = DMNSPDataset(test_df['before_utterances'].to_list(), test_df['dm'].to_list(), tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Model

In [21]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./llama3_finetuned",
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1000,
    fp16=True,
    push_to_hub=False
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

# Define LoRA configuration
peft_config = LoraConfig(
    r=8,  # Rank of the LoRA matrices
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to these layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA adapters
model = get_peft_model(model, peft_config)

In [23]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# Start training
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pongpak282546 (pongpak282546-chulalongkorn-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,No log,1.780827
200,No log,1.455397
300,No log,1.369251
400,No log,1.342334
500,1.675900,1.335595
600,1.675900,1.311022
700,1.675900,1.299994
800,1.675900,1.288918
900,1.675900,1.290850
1000,1.292500,1.277606


TrainOutput(global_step=10300, training_loss=1.2299458564832373, metrics={'train_runtime': 20519.7952, 'train_samples_per_second': 15418.916, 'train_steps_per_second': 963.704, 'total_flos': 1.9006052401938432e+18, 'train_loss': 1.2299458564832373, 'epoch': 0.5208596713021492})

In [24]:
output_dir = "best/"
model.save_pretrained(output_dir)

In [18]:
!ls

'D&Dpretrain.ipynb'					     llama3_finetuned
 HW08_LoRA_Student.ipynb				     midterm
 HW1_1_Dictionary_based_Tokenization_to_Student_2024.ipynb   project
 HW1_2_Neural_Network_Tokenization_to_Student_2024.ipynb     wandb


In [28]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
model = AutoModelForCausalLM.from_pretrained("llama3_finetuned/checkpoint-1500/",  load_in_4bit=True )

# Reapply LoRA adapters
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [104]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tmp = train_df['before_utterances'].to_list()

In [110]:
index = 3200
print(tmp[index])

<DM> <|begin_of_text|>Throwing chains up, some of them leap up and jab the actual hook into the side and try to pull back. Having a hard time piercing hide. It's real strong.<|end_of_text|>
<PLAYER7> <|begin_of_text|>Some of them can try to like swing the grapplings around the ankles--<|end_of_text|>
<PLAYER4> <|begin_of_text|>Shh, don't distract him, he's adding.<|end_of_text|>
<DM> <|begin_of_text|>I got this, don't worry.<|end_of_text|>
<PLAYER6> <|begin_of_text|>Don't piss him off.<|end_of_text|>


In [111]:
inputs = tokenizer(
            tmp[index],
            return_tensors='pt' ,
            truncation=True,
            max_length=256
        ).to(device)

In [112]:
out = model.generate(**inputs, max_length=300)

# Decode output tensor to text
output_text = tokenizer.decode(out[0], skip_special_tokens=True)
print(output_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<DM> Throwing chains up, some of them leap up and jab the actual hook into the side and try to pull back. Having a hard time piercing hide. It's real strong.
<PLAYER7> Some of them can try to like swing the grapplings around the ankles--
<PLAYER4> Shh, don't distract him, he's adding.
<DM> I got this, don't worry.
<PLAYER6> Don't piss him off.://
<DM>  I'll get him. He's a lot bigger than me.
<DM>  I'll get him. I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him.
<DM>  I'll get him


In [25]:
from google.colab import runtime
runtime.unassign()